In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/kaggle/input/ah-tickets/tickets.csv')

In [ ]:
data

In [ ]:
class Helper():
  def __init__(self,data):
    self.data = data
  def plot_data(self,save_path):
    num_rows = 2
    num_cols = 9
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 8))
    
    features = self.data.columns  # Assuming features are column names
    for i, feature in enumerate(features):
        row_index = i // num_cols
        col_index = i % num_cols

        unique_values = self.data[feature].unique()
        value_counts = self.data[feature].value_counts()

        axes[row_index, col_index].bar(unique_values, value_counts)
        axes[row_index, col_index].set_title(feature)
    if save_path:
        plt.savefig(save_path)

    plt.show()
    return features
  def drop_date(self):
    columns_to_drop = ['creation_date', 'view_date','action_date']
    self.data = self.data.drop(columns=columns_to_drop)
  def replace_values_to_number(self,column_name):
    list_of_values = self.data[column_name].unique()
    d = dict(zip(list_of_values, range(len(list_of_values))))
    for i in range(len(list_of_values)):
        self.data[column_name] = self.data[column_name].replace(list_of_values[i], i)
    return d
  def replace_values_to_number_with_dict(self,column_name, dic):
      list_of_values = self.data[column_name].unique()
      for i in range(len(list_of_values)):
          self.data[column_name] = self.data[column_name].replace(list_of_values[i], list(dic.keys())[i])
  def replace_dates(self):
    dates = ['creation','view','action']
    for date in dates:
        self.data[f'{date}_date'] = pd.to_datetime(self.data[f'{date}_date'])
        self.data[f'{date}_month'] = self.data[f'{date}_date'].dt.month
        self.data[f'{date}_day'] = self.data[f'{date}_date'].dt.day
        self.data[f'{date}_hour'] = self.data[f'{date}_date'].dt.hour
        self.data[f'{date}_minute'] = self.data[f'{date}_date'].dt.minute
  def convert_all_data_sig(self):
    features = self.data.head(0)
    thre = []
    for feature in features:
      thre.append(1/self.data[feature].max())
      self.data[feature]= self.data[feature]/self.data[feature].max()
    return thre
  def on_hot_encode(self):
    features = self.data.head(0)
    encoded_features=[]
    for feature in features:
      unique_values = data[feature].unique()
      encoded = np.eye(len(unique_values))[np.where(data[feature].values[:, None] == unique_values)[1]]
      encoded_features.append(encoded)
    return encoded_features

In [ ]:
H = Helper(data)
H.replace_dates()
H.drop_date()
tit = H.data.head(0)
dics = []
for col in tit:
    dics.append(H.replace_values_to_number(col))
cols = H.plot_data('/kaggle/working/real_data.png')
th_out = H.convert_all_data_sig()
data = H.data

In [ ]:
df_data = data

In [ ]:
data = tf.Variable(data)

In [ ]:
class GANS_Sigmoid():
  def __init__(self,data,noise_size,num_epochs,batch_size):
    self.data = data
    # Generate some random tabular data for demonstration purposes
    # feature = 1
    self.num_samples = self.data.shape[0]
    self.num_features = self.data.shape[1]
    self.noise_size = noise_size
    self.gen = self.generator()
    self.disc = self.discriminator()
    self.num_epochs = num_epochs
    self.batch_size = batch_size
# real_data = np.random.rand(num_samples, num_features)
  def Fit(self):
    disc_losses=[]
    gen_losses=[]
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)
    gen_vars = self.gen .trainable_variables
    disc_vars = self.disc.trainable_variables
    generator_optimizer.build(gen_vars)
    discriminator_optimizer.build(disc_vars)
    # Training loop
    num_epochs = self.num_epochs
    batch_size = self.batch_size
    for epoch in range(num_epochs):
        for _ in range(self.num_samples // batch_size):
            # Train the discriminator
            with tf.GradientTape() as tape:
                noise = tf.random.normal((batch_size, self.noise_size))
                fake_data = self.gen(noise)
                real_output = self.disc(data[_*batch_size:(_+1)*batch_size])
                fake_output = self.disc(fake_data)
                loss = self.discriminator_loss(real_output, fake_output)


                # # Build the optimizer with the model's variables

            gradients = tape.gradient(loss, self.disc.trainable_variables)
            # discriminator_optimizer.apply_gradients(zip(gradients, disc.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients, disc_vars))


            # Train the generator
            with tf.GradientTape() as tape:
                fake_data = self.gen(tf.random.normal((batch_size, self.noise_size)))
                fake_output = self.disc(fake_data)
                loss = self.generator_loss(fake_output)
            gradients = tape.gradient(loss, self.gen.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients, self.gen.trainable_variables))

        # if epoch % 100 == 0:
        disc_losses.append(np.mean(self.discriminator_loss(real_output, fake_output)))
        gen_losses.append(np.mean(self.generator_loss(fake_output)))
        print(f"Epoch {epoch}: Discriminator Loss = {np.mean(self.discriminator_loss(real_output, fake_output))}, Generator Loss = {np.mean(self.generator_loss(fake_output))}")
    return disc_losses,gen_losses
# Define the generator and discriminator networks
  def generator(self):
      model = tf.keras.Sequential([
          tf.keras.layers.Dense(128, activation='relu', input_shape=(self.noise_size,)),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(1024, activation='relu'),
#            tf.keras.layers.Dense(2024, activation='relu'),
#            tf.keras.layers.Dense(4048, activation='relu'),
#            tf.keras.layers.Dense(2024, activation='relu'),
#            tf.keras.layers.Dense(1024, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(self.num_features,activation='sigmoid')
      ])
      return model

  def discriminator(self):
      model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu', input_shape=(self.num_features,)),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
          # tf.keras.layers.Dense(1024, activation='relu'),
          # tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          # tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
      ])
      return model

# Define the loss function for the discriminator
  def discriminator_loss(self,real_output, fake_output):
      real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
      fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
      return real_loss + fake_loss

# Define the loss function for the generator
  def generator_loss(self,fake_output):
      return tf.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)
  def Predict(self,num_samples):
    output = self.gen(tf.random.normal((num_samples, self.noise_size)))
    final = np.zeros((num_samples,self.num_features))
    for i in range(self.num_features):
      final[:,i] = output[:,i]/th_out[i]
    final = np.round(final)
    return final
  def plot_dist(self, num_samples, feature_names, save_path=None):
    final = self.Predict(num_samples)
    
    # Calculate the number of rows and columns for the subplots
    num_rows = 2
    num_cols = 9
    
    # Create subplots
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 8))
    
    # Flatten the axes array for easier indexing
    axes = axes.flatten()
    
    for i in range(min(self.num_features, num_rows * num_cols)):
        unique_values, value_counts = np.unique(final[:, i], return_counts=True)
        axes[i].bar(unique_values, value_counts)
        axes[i].set_title(f'{feature_names[i]}')  # Use feature names as titles for each subplot

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)

    plt.show()




# Define the optimizers for the generator and discriminator


# Generate synthetic data with the trained generator
# synthetic_data = generator(tf.random.normal((num_samples, num_features)))




In [ ]:
G1 = GANS_Sigmoid(data,512,80,2048)
disc,gen = G1.Fit()

In [ ]:
plt.title('losses')
plt.plot(range(len(disc)),disc,label='Discriminator Loss',color='blue')
plt.plot(range(len(gen)),gen,label='Generator Loss',color='orange')
plt.legend()
plt.show()

In [ ]:
G1.plot_dist(306419,cols,'/kaggle/working/syn_data.png')

In [ ]:
Syn_sigmoid = G1.Predict(306419)

In [ ]:
df_syn = pd.DataFrame(data=Syn_sigmoid, columns=cols)

# Print the resulting DataFrame
df_syn

In [ ]:
G1.gen.save('/kaggle/working/model_sig')

In [ ]:
from scipy.stats import entropy

def compute_kl_divergence(original_data, synthesized_data):
    kl_divergence = 0.0
    all_values = set(original_data.values) | set(synthesized_data.values)

    for column in original_data.columns:
        # Compute probability distributions for each column
        p = original_data[column].value_counts(normalize=True).reindex(all_values, fill_value=0)
        q = synthesized_data[column].value_counts(normalize=True).reindex(all_values, fill_value=0)

        # Calculate KL divergence for the current column
        kl_divergence += entropy(p, q)

    # Aggregate KL divergence across all columns
    return kl_divergence

# Example usage
kl_divergence = compute_kl_divergence(df_data, df_syn)
print(f"KL Divergence: {kl_divergence}")


In [ ]:
data = pd.read_csv('/kaggle/input/ah-tickets/tickets.csv')

In [ ]:
H1 = Helper(data)
H1.replace_dates()
H1.drop_date()
tit = H1.data.head(0)
dics = []
for col in tit:
    dics.append(H1.replace_values_to_number(col))
H1.plot_data()
encoded = H1.on_hot_encode()

# data = H1.data

In [ ]:
class GANS_Soft():
  def __init__(self,data,noise_size,num_epochs,batch_size):
    self.data = data
    # Generate some random tabular data for demonstration purposes
    # feature = 1
    self.num_samples = self.data.shape[0]
    self.num_features = self.data.shape[1]
    self.noise_size = noise_size
    self.gen = self.generator()
    self.disc = self.discriminator()
    self.num_epochs = num_epochs
    self.batch_size = batch_size
# real_data = np.random.rand(num_samples, num_features)
  def Fit(self):
    disc_losses=[]
    gen_losses=[]
    generator_optimizer = tf.keras.optimizers.Adam(1e-2)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)
    gen_vars = self.gen .trainable_variables
    disc_vars = self.disc.trainable_variables
    generator_optimizer.build(gen_vars)
    discriminator_optimizer.build(disc_vars)
    # Training loop
    num_epochs = self.num_epochs
    batch_size = self.batch_size
    for epoch in range(num_epochs):
        for _ in range(self.num_samples // batch_size):
            # Train the discriminator
            with tf.GradientTape() as tape:
                noise = tf.random.normal((batch_size, self.noise_size))
                fake_data = self.gen(noise)
                real_output = self.disc(data[_*batch_size:(_+1)*batch_size])
                fake_output = self.disc(fake_data)
                loss = self.discriminator_loss(real_output, fake_output)


                # # Build the optimizer with the model's variables

            gradients = tape.gradient(loss, self.disc.trainable_variables)
            # discriminator_optimizer.apply_gradients(zip(gradients, disc.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients, disc_vars))


            # Train the generator
            with tf.GradientTape() as tape:
                fake_data = self.gen(tf.random.normal((batch_size, self.noise_size)))
                fake_output = self.disc(fake_data)
                loss = self.generator_loss(fake_output)
            gradients = tape.gradient(loss, self.gen.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients, self.gen.trainable_variables))

        # if epoch % 100 == 0:
        disc_losses.append(np.mean(self.discriminator_loss(real_output, fake_output)))
        gen_losses.append(np.mean(self.generator_loss(fake_output)))
        print(f"Epoch {epoch}: Discriminator Loss = {np.mean(self.discriminator_loss(real_output, fake_output))}, Generator Loss = {np.mean(self.generator_loss(fake_output))}")
    return disc_losses,gen_losses
# Define the generator and discriminator networks
  def generator(self):
      model = tf.keras.Sequential([
          tf.keras.layers.Dense(128, activation='relu', input_shape=(self.noise_size,)),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
#           tf.keras.layers.Dense(1024, activation='relu'),
#            tf.keras.layers.Dense(2024, activation='relu'),
#            tf.keras.layers.Dense(4048, activation='relu'),
#            tf.keras.layers.Dense(2024, activation='relu'),
           tf.keras.layers.Dense(1024, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(self.num_features,activation='softmax')
      ])
      # fake_data = model(input_noise)
      return model

  def discriminator(self):
      model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu', input_shape=(self.num_features,)),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(512, activation='relu'),
          # tf.keras.layers.Dense(1024, activation='relu'),
          # tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          # tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
      ])
      # output = model(input_data)
      return model

# Define the loss function for the discriminator
  def discriminator_loss(self,real_output, fake_output):
      real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
      fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
      return real_loss + fake_loss

# Define the loss function for the generator
  def generator_loss(self,fake_output):
      return tf.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)
  def Predict(self,num_samples):
    output = self.gen(tf.random.normal((num_samples, self.noise_size)))
    final = np.zeros((num_samples,self.num_features))
    for i in range(self.num_features):
      final[:,i] = output[:,i]/th_out[i]
    final = np.round(final)
    return final
  def plot_dist(self, num_samples, feature_names, save_path=None):
    final = self.Predict(num_samples)
    
    # Calculate the number of rows and columns for the subplots
    num_rows = 2
    num_cols = 9
    
    # Create subplots
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 8))
    
    # Flatten the axes array for easier indexing
    axes = axes.flatten()
    
    for i in range(min(self.num_features, num_rows * num_cols)):
        unique_values, value_counts = np.unique(final[:, i], return_counts=True)
        axes[i].bar(unique_values, value_counts)
        axes[i].set_title(f'{feature_names[i]}')  # Use feature names as titles for each subplot

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)

    plt.show()


In [ ]:
for i in raneg(len(encoded)):
    G2 = GANS_Soft(encoded[i],100,20,2048)
    disc,gen = G2.Fit()
    G2.gen.save(f'/kaggle/working/model_sofmax{}')
    G2.plot_dist(306419,'')

In [ ]:
# fin = G1.Predict(306419)
G1.plot_dist(306419)